In [58]:
import pandas as pd

In [205]:
data = pd.read_csv('/Users/ilchenkoslava/PycharmProjects/Upwork/MLSN/mlsn_property_data.csv')

In [130]:
data.head().T

,0,1,2,3,4
адрес,ул. 3-я Любинская 28а,ул. 3-я Любинская 28а,ул. 3-я Любинская 28а,ул. Ватутина 8,ул. Перелета 19
этаж,2/5,4/5,4/5,2/16,3/12
этажность,-,-,-,-,-
площадь общая,35.2 м²,35.2 м²,40 м²,38.5 м²,42.5 м²
площадь жилая,17.5 м²,17.5 м²,19.7 м²,16.6 м²,14.4 м²
площадь кухни,9.5 м²,9.5 м²,12.1 м²,8.7 м²,13.3 м²
планировка,свободная планировка,свободная планировка,свободная планировка,свободная планировка,свободная планировка
санузел,совмещенный,совмещенный,совмещенный,совмещенный,совмещенный
балкон,балкон,балкон,балкон,лоджия,балкон и лоджия
состояние/ремонт,черновая отделка,черновая отделка,черновая отделка,черновая отделка,черновая отделка


In [115]:
data.columns

Index(['адрес', 'этаж', 'этажность', 'площадь общая', 'площадь жилая',
       'площадь кухни', 'планировка', 'санузел', 'балкон', 'состояние/ремонт',
       'покрытие пола', 'материал окон', 'входная дверь', 'вид из окон',
       'угловая', 'домофон', 'счетчики воды', 'тип дома', 'материал дома',
       'новостройка', 'срок сдачи', 'высота потолка', 'интернет', 'лифт',
       'мусоропровод', 'парковка', 'рейтинг', 'экология', 'инфраструктура',
       'безопасность', 'социальное восприятие', 'уровень комфорта',
       'транспортная доступность', 'цена'],
      dtype='object')

In [64]:
data.shape

(611, 33)

In [206]:
data.insert(2, 'этажность', '-')

In [207]:
data.columns = data.apply(lambda x: x.name.strip())

In [208]:
data['этажность'] = data['этаж'].apply(lambda x: x.split('/')[-1])
data['этаж'] = data['этаж'].apply(lambda x: x.split('/')[0])

In [219]:
data['площадь общая'] = data['площадь общая'].apply(lambda x: x.replace('\xa0м²', ''))
data['площадь жилая'] = data['площадь жилая'].apply(lambda x: x.replace('\xa0м²', ''))
data['площадь кухни'] = data['площадь кухни'].apply(lambda x: x.replace('\xa0м²', ''))

In [220]:
data.head().T

,0,1,2,3,4
адрес,ул. 3-я Любинская 28а,ул. 3-я Любинская 28а,ул. 3-я Любинская 28а,ул. Ватутина 8,ул. Перелета 19
этаж,2,4,4,2,3
этажность,5,5,5,16,12
площадь общая,35.2,35.2,40,38.5,42.5
площадь жилая,17.5,17.5,19.7,16.6,14.4
площадь кухни,9.5,9.5,12.1,8.7,13.3
планировка,свободная планировка,свободная планировка,свободная планировка,свободная планировка,свободная планировка
санузел,совмещенный,совмещенный,совмещенный,совмещенный,совмещенный
балкон,балкон,балкон,балкон,лоджия,балкон и лоджия
состояние/ремонт,черновая отделка,черновая отделка,черновая отделка,черновая отделка,черновая отделка


In [221]:
from sklearn.preprocessing import LabelEncoder